# Análisis de precios de apartamentos de Bogotá

# Importacion de librerías y funciones auxiliares

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import itertools

In [85]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import root_mean_squared_error
from sklearn.linear_model import LinearRegression, Ridge

In [4]:
df = pd.read_csv('C:/Users/jmriv/Documents/Ciencia de Datos - Repos/Tutorial regresión/apartamentos.csv')

In [5]:
def ver_datos(data, col_name, categorica=False):
    print(data[col_name].describe())
    print('--'*10)
    print(f'Número de datos nulos: {data[col_name].isnull().sum()}')
    if categorica:
        print('--'*10)
        print(data[col_name].value_counts(dropna=False))

# Caracterización de los datos

In [6]:
# Información básica del dataset
print('Dimensiones del dataset (filas, columnas):', df.shape)
print('--'*10)
print('Tipos de datos:')
display(df.info())

Dimensiones del dataset (filas, columnas): (43013, 46)
--------------------
Tipos de datos:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43013 entries, 0 to 43012
Data columns (total 46 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   _id                      43013 non-null  object 
 1   codigo                   43013 non-null  object 
 2   tipo_propiedad           43013 non-null  object 
 3   tipo_operacion           43013 non-null  object 
 4   precio_venta             27584 non-null  float64
 5   area                     43013 non-null  float64
 6   habitaciones             43012 non-null  float64
 7   banos                    43012 non-null  float64
 8   administracion           35088 non-null  float64
 9   parqueaderos             43012 non-null  float64
 10  sector                   41372 non-null  object 
 11  estrato                  43012 non-null  float64
 12  antiguedad               43003 non-nul

None

In [110]:
print(df['precio_venta'].mean())
print(df['precio_venta'].std())

2365108333.133048
55757129237.449104


In [7]:
print('Estadísticas descriptivas:')
print(df.describe())

Estadísticas descriptivas:
       precio_venta          area  habitaciones         banos  administracion  \
count  2.758400e+04  4.301300e+04  43012.000000  43012.000000    3.508800e+04   
mean   2.364794e+09  1.800341e+02      2.615340      2.701339    4.059551e+06   
std    5.574198e+10  9.163002e+03      0.850046      1.196659    6.781642e+07   
min    1.000000e+06  0.000000e+00      1.000000      0.000000    1.000000e+00   
25%    4.029000e+08  6.300000e+01      2.000000      2.000000    3.490000e+05   
50%    7.000000e+08  1.000000e+02      3.000000      3.000000    6.500000e+05   
75%    1.330000e+09  1.740000e+02      3.000000      4.000000    1.200000e+06   
max    4.250000e+12  1.900000e+06      7.000000      6.000000    3.500000e+09   

       parqueaderos       estrato       latitud      longitud  \
count  43012.000000  43012.000000  43013.000000  43013.000000   
mean       1.662862      4.844648      4.686099    -74.062808   
std        1.114685      1.236190      0.038297 

Se identifica que la variable _Parqueaderos_ tiene una entrada negativa, por lo que se revisará su validez

In [8]:
df.sample(2)

,_id,codigo,tipo_propiedad,tipo_operacion,precio_venta,area,habitaciones,banos,administracion,parqueaderos,...,vigilancia,coords_modified,localidad,barrio,estacion_tm_cercana,distancia_estacion_tm_m,is_cerca_estacion_tm,parque_cercano,distancia_parque_m,is_cerca_parque
33123,66d86de9ceda690e855108c3,11079-M5065878,APARTAMENTO,ARRIENDO,NaN,130.0,3.0,2.0,1000000.0,2.0,...,0.0,False,CHAPINERO,S.C. CHICO NORTE III SECTOR,Calle 100 - Marketmedios,782.17,0,PARQUE VECINAL URBANIZACIÓN NORMANDIA I II II ...,973.78,0
16257,66d86cedceda690e8550c6e1,4278-M5023415,APARTAMENTO,ARRIENDO,NaN,70.0,2.0,3.0,523000.0,1.0,...,1.0,False,USAQUEN,BELLA SUIZA,Calle 127,2685.39,0,PARQUE METROPOLITANO EL COUNTRY,945.01,0


In [9]:
# Información básica del dataset
print('Dimensiones del dataset (filas, columnas):', df.shape)
print('\nTipos de datos:')
print(df.dtypes)
print('\nEstadísticas descriptivas:')
print(df.describe())

Dimensiones del dataset (filas, columnas): (43013, 46)

Tipos de datos:
_id                         object
codigo                      object
tipo_propiedad              object
tipo_operacion              object
precio_venta               float64
area                       float64
habitaciones               float64
banos                      float64
administracion             float64
parqueaderos               float64
sector                      object
estrato                    float64
antiguedad                  object
latitud                    float64
longitud                   float64
direccion                   object
descripcion                 object
website                     object
last_view                   object
datetime                    object
url                         object
timeline                    object
estado                      object
compañia                    object
precio_arriendo            float64
jacuzzi                    float64
piso              

In [10]:
df['tipo_operacion'].value_counts()

tipo_operacion
VENTA               27270
ARRIENDO            15515
VENTA Y ARRIENDO      228
Name: count, dtype: int64

In [11]:
df[['tipo_propiedad','tipo_operacion']].value_counts()

tipo_propiedad             tipo_operacion  
APARTAMENTO                VENTA               27200
                           ARRIENDO            15515
                           VENTA Y ARRIENDO      228
CASA CON CONJUNTO CERRADO  VENTA                  60
CASA                       VENTA                  10
Name: count, dtype: int64

## Validez

In [12]:
# Se asume que el valor negativo de los parqueaderos puede ser un typo
df.loc[df['parqueaderos'] < 0, 'parqueaderos'] = 2

In [13]:
# Se asume que los inmuebles con 10, 20 y 30 parqueaderos también fueron typos
df['parqueaderos'].value_counts()

parqueaderos
2.0     15526
1.0     12678
0.0      6615
3.0      5045
4.0      3143
10.0        2
20.0        2
30.0        1
Name: count, dtype: int64

In [14]:
df['parqueaderos'] = df['parqueaderos'].replace({10:1,20:2,30:3})

In [15]:
df['banos'].value_counts()

banos
2.0    14825
3.0    10605
4.0     6615
1.0     6614
5.0     4329
0.0       23
6.0        1
Name: count, dtype: int64

In [16]:
# Se considera como no valido los inmuebles que no tiene baños
df = df[df['banos'] > 0]

In [17]:
# Dado que no hay ningún inmueble con salón comunal, se excluye del análisis
df['salon_comunal'].value_counts()

salon_comunal
0.0    42987
Name: count, dtype: int64

In [18]:
# Dado que no hay ningún inmueble que permita mascotas, se excluye del análisis
df['permite_mascotas'].value_counts()

permite_mascotas
0.0    42987
Name: count, dtype: int64

In [19]:
# Dado que no hay ningún inmueble que tenga chimenea, se excluye del análisis
df['chimenea'].value_counts()

chimenea
0.0    42987
Name: count, dtype: int64

In [20]:
# Se valida que todos los inmuebles tengan al menos un precio asociado
df.loc[(df["precio_arriendo"].isna()) & (df["precio_venta"].isna())]

,_id,codigo,tipo_propiedad,tipo_operacion,precio_venta,area,habitaciones,banos,administracion,parqueaderos,...,vigilancia,coords_modified,localidad,barrio,estacion_tm_cercana,distancia_estacion_tm_m,is_cerca_estacion_tm,parque_cercano,distancia_parque_m,is_cerca_parque
15148,66d86ce1ceda690e8550c28c,4058-M5037036,APARTAMENTO,ARRIENDO,NaN,350.0,3.0,5.0,NaN,3.0,...,1.0,False,USAQUEN,LISBOA_,Prado,2346.01,0,PARQUE VECINAL NUEVA URBANIZACIÓN EL CEDRITO,1123.48,0
24076,66d86d50ceda690e8550e56c,4058-M5044663,APARTAMENTO,ARRIENDO,NaN,220.0,3.0,2.0,NaN,1.0,...,0.0,False,USAQUEN,LISBOA_,Alcalá – Colegio S. Tomás Dominicos,1481.35,0,PARQUE VECINAL NUEVA URBANIZACIÓN EL CEDRITO,950.64,0
34333,66d86e01ceda690e85510d7d,4058-M5068686,APARTAMENTO,ARRIENDO,NaN,220.0,3.0,2.0,NaN,1.0,...,0.0,False,USAQUEN,LISBOA_,Alcalá – Colegio S. Tomás Dominicos,1481.35,0,PARQUE VECINAL NUEVA URBANIZACIÓN EL CEDRITO,950.64,0


In [21]:
df = df.loc[
    (df["precio_arriendo"].notna())
    | (df["precio_venta"].notna())]

## Completitud
Se eliminarán algunas variables teniendo en cuenta el número de nulos que presentan

In [22]:
print('\nValores nulos por columna:')
print(df.isnull().sum().sort_values(ascending=False))


Valores nulos por columna:
closets                    42986
piso                       42986
url                        42193
direccion                  42193
precio_arriendo            27167
precio_venta               15417
timeline                   13580
administracion              7917
compañia                    4529
sector                      1638
estado                       795
barrio                       193
antiguedad                    10
website                        2
salon_comunal                  2
terraza                        2
conjunto_cerrado               2
ascensor                       2
vigilancia                     2
piscina                        2
gimnasio                       2
datetime                       2
chimenea                       2
permite_mascotas               2
last_view                      2
jacuzzi                        2
estrato                        1
_id                            0
tipo_operacion                 0
area           

Se deciden excluir las siguientes variables debido al número de datos nulos que presentan:
- Closets
- Piso
- URL
- Dirección

In [23]:
df.drop(columns=['closets', 'piso','url','direccion'], inplace=True)

## Relevancia
Se decide seleccionar un subconjunto de los datos para trabajar con ellos

In [24]:
selected_cols = [
    'precio_arriendo',
    'precio_venta',
    'terraza',
    'vigilancia',
    'piscina',
    'ascensor',
    'conjunto_cerrado',
    'gimnasio',
    'jacuzzi',
    'estrato',
    'habitaciones',
    'banos',
    'parqueaderos',
    'tipo_operacion',
    'tipo_propiedad',
    'area',
    'localidad',
    'distancia_estacion_tm_m',
    'is_cerca_estacion_tm',
    'distancia_parque_m',
    'is_cerca_parque',
    #'antiguedad',
    ]

In [25]:
df_selected=df[selected_cols]

In [26]:
print(df_selected.isnull().sum().sort_values(ascending=False))

precio_arriendo            27167
precio_venta               15417
terraza                        2
vigilancia                     2
piscina                        2
ascensor                       2
conjunto_cerrado               2
gimnasio                       2
jacuzzi                        2
estrato                        1
habitaciones                   0
banos                          0
parqueaderos                   0
tipo_operacion                 0
tipo_propiedad                 0
area                           0
localidad                      0
distancia_estacion_tm_m        0
is_cerca_estacion_tm           0
distancia_parque_m             0
is_cerca_parque                0
dtype: int64


# Divisón de los datos

Dado que el modelo está pensado para apartamentos en venta, el conjunto de test será exclusivamente de inmuevles tipo **APARTAMENTO** y de operaciones tipo **VENTA** y **VENTA Y ARRIENDO**

In [91]:
# Filtar el dataframe para inmuebles tipo APARTAMENTO y operacion VENTA o VENTA Y ARRIENDO
df_aptos_venta = df_selected.loc[
    (df["tipo_propiedad"]=="APARTAMENTO")
    & (df["tipo_operacion"].isin(['VENTA','VENTA Y ARRIENDO']))
    ]

In [92]:
df_aptos_venta
print(f'El dataset tiene {df_aptos_venta.shape[0]} entradas antes de eliminar nulos')
df_aptos_venta = df_aptos_venta.dropna(subset=df_aptos_venta.columns[1:])
print(f'El dataset tiene {df_aptos_venta.shape[0]} entradas después de eliminar nulos')

El dataset tiene 27414 entradas antes de eliminar nulos
El dataset tiene 27411 entradas después de eliminar nulos


In [93]:
X_aptos_venta = df_aptos_venta.drop(columns=['precio_venta'])
y_aptos_venta = df_aptos_venta['precio_venta']
X_train, X_test, y_train, y_test = train_test_split(X_aptos_venta, y_aptos_venta, test_size=0.30, random_state=42)

# Pre-procesamiento para modelado

In [30]:
# Se toma el conjunto opuesto a apartamentos en venta, del que se tomó el conjunto de test
df_resto = df_selected.loc[
    (df["tipo_propiedad"]!="APARTAMENTO")
    | (df["tipo_operacion"].isin(['ARRIENDO']))
    ]
df_resto.shape

(15572, 21)

In [31]:
print(df_resto.isnull().sum().sort_values(ascending=False))

precio_venta               15417
precio_arriendo               70
terraza                        0
vigilancia                     0
piscina                        0
ascensor                       0
conjunto_cerrado               0
gimnasio                       0
jacuzzi                        0
estrato                        0
habitaciones                   0
banos                          0
parqueaderos                   0
tipo_operacion                 0
tipo_propiedad                 0
area                           0
localidad                      0
distancia_estacion_tm_m        0
is_cerca_estacion_tm           0
distancia_parque_m             0
is_cerca_parque                0
dtype: int64


In [32]:
df_train = X_train.copy()
df_train['precio_venta'] = y_train
print(df_train.isnull().sum().sort_values(ascending=False))

precio_arriendo            18954
terraza                        0
vigilancia                     0
piscina                        0
ascensor                       0
conjunto_cerrado               0
gimnasio                       0
jacuzzi                        0
estrato                        0
habitaciones                   0
banos                          0
parqueaderos                   0
tipo_operacion                 0
tipo_propiedad                 0
area                           0
localidad                      0
distancia_estacion_tm_m        0
is_cerca_estacion_tm           0
distancia_parque_m             0
is_cerca_parque                0
precio_venta                   0
dtype: int64


In [33]:
df_full = pd.concat([df_train, df_resto])
df_full

,precio_arriendo,terraza,vigilancia,piscina,ascensor,conjunto_cerrado,gimnasio,jacuzzi,estrato,habitaciones,...,parqueaderos,tipo_operacion,tipo_propiedad,area,localidad,distancia_estacion_tm_m,is_cerca_estacion_tm,distancia_parque_m,is_cerca_parque,precio_venta
25738,NaN,0.0,1.0,0.0,0.0,1.0,0.0,0.0,3.0,3.0,...,1.0,VENTA,APARTAMENTO,66.0,USAQUEN,529.65,0,708.60,0,3.000000e+08
29093,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,2.0,...,2.0,VENTA,APARTAMENTO,80.0,USAQUEN,1821.52,0,931.93,0,4.500000e+08
3622,NaN,0.0,0.0,0.0,0.0,1.0,0.0,0.0,3.0,2.0,...,0.0,VENTA,APARTAMENTO,38.0,SUBA,719.86,0,647.21,0,1.900000e+08
41254,NaN,0.0,1.0,0.0,1.0,1.0,1.0,0.0,6.0,3.0,...,2.0,VENTA,APARTAMENTO,220.0,USAQUEN,2849.07,0,932.10,0,1.800000e+09
31184,NaN,0.0,0.0,0.0,0.0,1.0,0.0,0.0,4.0,3.0,...,1.0,VENTA,APARTAMENTO,60.0,FONTIBON,2680.80,0,1077.17,0,3.100000e+08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41226,NaN,0.0,0.0,0.0,0.0,1.0,0.0,0.0,4.0,3.0,...,1.0,VENTA,CASA CON CONJUNTO CERRADO,88.0,SUBA,1191.34,0,485.24,1,3.940000e+08
41250,NaN,0.0,0.0,0.0,0.0,1.0,0.0,0.0,2.0,2.0,...,0.0,VENTA,CASA CON CONJUNTO CERRADO,44.0,BOSA,3276.84,0,332.08,1,1.470000e+08
41251,NaN,0.0,0.0,0.0,0.0,1.0,0.0,0.0,2.0,2.0,...,0.0,VENTA,CASA CON CONJUNTO CERRADO,54.0,KENNEDY,1673.59,0,835.92,0,1.785000e+08
42930,NaN,1.0,0.0,0.0,0.0,1.0,0.0,0.0,2.0,3.0,...,0.0,VENTA,CASA CON CONJUNTO CERRADO,68.0,KENNEDY,1284.54,0,350.58,1,2.510000e+08


In [34]:
df_long = df_full.melt(
    id_vars=[c for c in df_selected.columns if c not in ["precio_venta", "precio_arriendo"]],
    value_vars=["precio_venta", "precio_arriendo"],
    var_name="es_precio_venta",
    value_name="precio"
).dropna(subset=["precio"])
df_long["es_precio_venta"] = df_long["es_precio_venta"] == "precio_venta"
df_long["es_apartamento"] = df_long["tipo_propiedad"] == "APARTAMENTO"
df_long = df_long.drop(columns=['tipo_operacion','tipo_propiedad'])

In [35]:
print(df_long.isnull().sum().sort_values(ascending=False))

terraza                    0
vigilancia                 0
piscina                    0
ascensor                   0
conjunto_cerrado           0
gimnasio                   0
jacuzzi                    0
estrato                    0
habitaciones               0
banos                      0
parqueaderos               0
area                       0
localidad                  0
distancia_estacion_tm_m    0
is_cerca_estacion_tm       0
distancia_parque_m         0
is_cerca_parque            0
es_precio_venta            0
precio                     0
es_apartamento             0
dtype: int64


## Preparación de variables categóricas

Se incluyeron dos columnas categóricas como parte del análisis:
- Localidad
- Antigüedad

Para la variable _localidad_ se dividirá la variable en columnas dummy

Para la variable _antigüedad_ se propone convertirla en 3 variables:
 - Nuevo: incluye las variables _SOBRE PLANOS_, _EN CONSTRUCCION_ y _PARA ESTRENAR_
 - Usado: incluye las variables _ENTRE 0 Y 5 ANOS_ hasta _MAS DE 20 ANOS_
 - Remodelado: incluye la variable _REMODELADO_

In [36]:
df['antiguedad'].value_counts()

antiguedad
MAS DE 20 ANOS        14349
ENTRE 10 Y 20 ANOS    11525
ENTRE 0 Y 5 ANOS       8047
ENTRE 5 Y 10 ANOS      7018
REMODELADO             1811
SOBRE PLANOS            107
EN CONSTRUCCION         101
PARA ESTRENAR            18
Name: count, dtype: int64

In [37]:
#plt.figure(figsize=(12,4))
#ax = sns.boxplot(
#    data=df_clean,
#    x='antiguedad',
#    y='precio_venta',
#    showmeans=True
#)
#plt.ylim(0,3000000000)

# Modelado

1. crear el traint, test y validation
2. Crear al menos dos modelos distintos para predecir el precio_venta
3. Metricas
4. Elegir un modelo y jsutificarlo, oportunidades de mejora
5. interpretabilidad shapley line 
6. generacion de valor 
7. insights

In [63]:
df_long["es_precio_venta"].value_counts()

es_precio_venta
True     19342
False    15735
Name: count, dtype: int64

In [39]:
df_long["es_apartamento"].value_counts()

es_apartamento
True     35007
False       70
Name: count, dtype: int64

In [72]:
def entrenar_modelos(df_input, solo_aptos, localidad, solo_venta, escalado_estandar, remover_atipicos, modelo, hiperparametros):
    df = df_input.copy()
    
    if solo_aptos:
        df = df.loc[(df["es_apartamento"] == solo_aptos)]
    else:
        df = df.drop(columns=["es_apartamento"])

    if solo_venta:
        df = df.loc[(df["es_precio_venta"] == solo_venta)]
    else:
        df = df.drop(columns=["es_precio_venta"])
    
    print(f'El dataset tiene {df.shape[0]} entradas antes de eliminar nulos')
    df=df.dropna()
    print(f'El dataset tiene {df.shape[0]} entradas después de eliminar nulos')
    
    if localidad:
        df = pd.get_dummies(df, columns=["localidad"])
    else:
        df = df.drop(columns=["localidad"])

    if remover_atipicos:
        df = df[df['precio'] > df['precio'].quantile(0.01)]
        df = df[df['precio'] < df['precio'].quantile(0.99)]
        print(f'El dataset tiene {df.shape[0]} entradas después de eliminar valores atípicos')

    X = df.drop(columns=['precio'])
    y = df['precio']

    scaler = None
    if escalado_estandar:
        scaler = StandardScaler()
        X = scaler.fit_transform(X)
    
    search = GridSearchCV(
        estimator=modelo,
        param_grid=hiperparametros,
        cv=20,
        scoring='neg_root_mean_squared_error'
    )

    search.fit(X, y)

    return search, scaler
    

### Regresión lineal

In [96]:
combinations = list(itertools.product([True, False], repeat=5))

In [99]:
best_linear_results = {}

param_grid_lr = {
    "fit_intercept":[True]
}

i=0
best_score=10000000000000
best_combo=""
best_comb_index=-1
for comb in combinations:
    description = f"Modelo {i}: {'APARTAMENTOS' if comb[0] else 'APARTAMENTOS Y CASAS'} | {'LOCALIDADES' if comb[1] else 'SIN LOCALIDADES'} | {'SOLO VENTA' if comb[2] else 'VENTA Y ARRIENDO'} | {'STD SCALER' if comb[3] else 'NO SCALER'} | {'SIN ATIPICOS' if comb[4] else 'CON ATIPICOS'}"
    print(description)
    result, scaler = entrenar_modelos(
        df_input=df_long,
        solo_aptos=comb[0],
        localidad=comb[1],
        solo_venta=comb[2],
        escalado_estandar=comb[3],
        remover_atipicos=comb[4],
        modelo = LinearRegression(),
        hiperparametros = param_grid_lr)
    print(result.best_params_)
    print(-result.best_score_)
    best_linear_results[i]=[result, scaler]

    if -result.best_score_ < best_score:
        best_score = -result.best_score_
        best_combo=description
        best_comb_index=i
    print('--'*20)
    i+=1

Modelo 0: APARTAMENTOS | LOCALIDADES | SOLO VENTA | STD SCALER | SIN ATIPICOS
El dataset tiene 19272 entradas antes de eliminar nulos
El dataset tiene 19272 entradas después de eliminar nulos
El dataset tiene 18887 entradas después de eliminar valores atípicos
{'fit_intercept': True}
493561012.2134404
----------------------------------------
Modelo 1: APARTAMENTOS | LOCALIDADES | SOLO VENTA | STD SCALER | CON ATIPICOS
El dataset tiene 19272 entradas antes de eliminar nulos
El dataset tiene 19272 entradas después de eliminar nulos
{'fit_intercept': True}
40967616491.82635
----------------------------------------
Modelo 2: APARTAMENTOS | LOCALIDADES | SOLO VENTA | NO SCALER | SIN ATIPICOS
El dataset tiene 19272 entradas antes de eliminar nulos
El dataset tiene 19272 entradas después de eliminar nulos
El dataset tiene 18887 entradas después de eliminar valores atípicos
{'fit_intercept': True}
493561012.21341527
----------------------------------------
Modelo 3: APARTAMENTOS | LOCALIDADES 

In [100]:
print(best_score)
print(best_combo)
print(best_comb_index)

492759836.6475294
Modelo 18: APARTAMENTOS Y CASAS | LOCALIDADES | SOLO VENTA | NO SCALER | SIN ATIPICOS
18


### Regresión lineal con regularización de Ridge

In [101]:
combinations = list(itertools.product([True, False], repeat=4))

In [102]:
best_ridge_results = {}

param_grid_ridge = {
    "alpha": [0.01, 0.1, 1, 10, 100, 300, 1000]
}

i=0
best_score=10000000000000
best_combo=""
best_comb_index=-1
for comb in combinations:
    description = f"Modelo {i}: {'APARTAMENTOS' if comb[0] else 'APARTAMENTOS Y CASAS'} | {'LOCALIDADES' if comb[1] else 'SIN LOCALIDADES'} | {'SOLO VENTA' if comb[2] else 'VENTA Y ARRIENDO'} | {'STD SCALER' if True else 'NO SCALER'} | {'SIN ATIPICOS' if comb[3] else 'CON ATIPICOS'}"
    print(description)
    result, scaler = entrenar_modelos(
        df_input=df_long,
        solo_aptos=comb[0],
        localidad=comb[1],
        solo_venta=comb[2],
        escalado_estandar=True,
        remover_atipicos=comb[3],
        modelo = Ridge(),
        hiperparametros = param_grid_ridge)
    print(result.best_params_)
    print(-result.best_score_)
    best_ridge_results[i]=[result, scaler]

    if -result.best_score_ < best_score:
        best_score = -result.best_score_
        best_combo=description
        best_comb_index=i
    i+=1
    print('--'*20)

Modelo 0: APARTAMENTOS | LOCALIDADES | SOLO VENTA | STD SCALER | SIN ATIPICOS
El dataset tiene 19272 entradas antes de eliminar nulos
El dataset tiene 19272 entradas después de eliminar nulos
El dataset tiene 18887 entradas después de eliminar valores atípicos


{'alpha': 100}
493551101.7531885
----------------------------------------
Modelo 1: APARTAMENTOS | LOCALIDADES | SOLO VENTA | STD SCALER | CON ATIPICOS
El dataset tiene 19272 entradas antes de eliminar nulos
El dataset tiene 19272 entradas después de eliminar nulos
{'alpha': 1000}
40886808149.699814
----------------------------------------
Modelo 2: APARTAMENTOS | LOCALIDADES | VENTA Y ARRIENDO | STD SCALER | SIN ATIPICOS
El dataset tiene 35007 entradas antes de eliminar nulos
El dataset tiene 35007 entradas después de eliminar nulos
El dataset tiene 34292 entradas después de eliminar valores atípicos
{'alpha': 1000}
682908704.8454006
----------------------------------------
Modelo 3: APARTAMENTOS | LOCALIDADES | VENTA Y ARRIENDO | STD SCALER | CON ATIPICOS
El dataset tiene 35007 entradas antes de eliminar nulos
El dataset tiene 35007 entradas después de eliminar nulos
{'alpha': 1000}
26264324376.36379
----------------------------------------
Modelo 4: APARTAMENTOS | SIN LOCALIDADES | 

In [103]:
print(best_score)
print(best_combo)
print(best_comb_index)

492750797.907924
Modelo 8: APARTAMENTOS Y CASAS | LOCALIDADES | SOLO VENTA | STD SCALER | SIN ATIPICOS
8


## Entrenamiento y evaluación con test

In [104]:
# Modelo: APARTAMENTOS Y CASAS | LOCALIDADES | SOLO VENTA | NO SCALER | SIN ATIPICOS
best_linreg = best_linear_results[18][0].best_estimator_
best_linreg_scaler = best_linear_results[18][1]

# Modelo: APARTAMENTOS Y CASAS | LOCALIDADES | SOLO VENTA | STD SCALER | SIN ATIPICOS
best_ridge = best_ridge_results[8][0].best_estimator_
best_ridge_scaler = best_ridge_results[8][1]

Vamos a preparar el conjunto de test para que sea compatible con los modelos entrenados

In [105]:
def test_modelos(X, y, scaler, solo_aptos, localidad, solo_venta, escalado_estandar, modelo):
    df = X.copy()
    
    df = df.drop(columns=['precio_arriendo','tipo_operacion','tipo_propiedad'])
    df['es_precio_venta'] = True
    df['es_apartamento'] = True

    if not solo_aptos:
        df = df.drop(columns=["es_apartamento"])

    if not solo_venta:
        df = df.drop(columns=["es_precio_venta"])
    
    if localidad:
        df = pd.get_dummies(df, columns=["localidad"])
    else:
        df = df.drop(columns=["localidad"])    

    if escalado_estandar:
        df = scaler.transform(df)

    y_pred = modelo.predict(df)
    rmse = root_mean_squared_error(y, y_pred)

    print("Test RMSE:", rmse)

In [106]:
test_modelos(X_test, y_test, best_linreg_scaler, False, True, True, False, best_linreg)

Test RMSE: 33963871303.107735


In [107]:
test_modelos(X_test, y_test, best_ridge_scaler, False, True, True, True, best_ridge)

Test RMSE: 33963947537.45847


Se probaron diferentes pasos de preprocesamiento y se comparó su capacidad para predecir el precio de los inmuebles mediante el error promedio cuadrado (RMSE). Se probaron dos modelos: una regresión lineal y una regresión lineal con normalización de Ridge.

Así, los modelos elegidos fueron los siguientes

| Tipo de inmueble       | incluye localidad (dummy) | Incluye antigüedad | Tipo de operación | ¿Escalado?  | Modelo           | Remover atípicos | Average RMSE                             | Test RMSE                              |
|------------------------|---------------------------|--------------------|-------------------|-------------|------------------|------------------|------------------------------------------|----------------------------------------|
| Apartamentos y   casas | Sí                        | No                 | Solo venta        | No escalado | Regresión lineal | Sí               |  $                       492.759.836,65  |   $ 33.963.871.303,11 |
| Apartamentos y casas   | Sí                        | No                 | Solo venta        | Standard    | Ridge            | Sí               |  $                       492.750.797,91  |   $ 33.963.947.537,46 |
